In [71]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
import getpass
import pymysql
#import matplotlib.pyplot as plt
#import geoplot as gplt
import folium

p = getpass.getpass()
connection = pymysql.connect(host = 'localhost', port = 3306, user = 'root', passwd = p) #, db='mysql')
#connection.autocommit(True)
cursor = connection.cursor()

In [2]:
def geometrify(wkt):
    if wkt is None:
        return np.nan
    else:
        return shapely.wkt.loads(wkt)

cursor.execute('use geo_analysis;')
brazil = pd.read_sql("SELECT id, UF, name, ST_AsText(geometry) AS geometry FROM cities;", con = connection)

brazil['geometry'] = brazil['geometry'].map(geometrify)
brazil = gpd.GeoDataFrame(brazil)
brazil

,id,UF,name,geometry
0,1100015,RO,Alta Floresta D'Oeste,"POLYGON ((-62.18209 -11.86686, -62.16230 -11.8..."
1,1100023,RO,Ariquemes,"POLYGON ((-62.53595 -9.73182, -62.50782 -9.754..."
2,1100031,RO,Cabixi,"POLYGON ((-60.39940 -13.45584, -60.40195 -13.4..."
3,1100049,RO,Cacoal,"POLYGON ((-61.00051 -11.39796, -61.01794 -11.4..."
4,1100056,RO,Cerejeiras,"POLYGON ((-61.50047 -13.00392, -61.47901 -13.0..."
...,...,...,...,...
5559,5222005,GO,Vianópolis,"POLYGON ((-48.43125 -16.62755, -48.42527 -16.6..."
5560,5222054,GO,Vicentinópolis,"POLYGON ((-49.85005 -17.57682, -49.84311 -17.5..."
5561,5222203,GO,Vila Boa,"POLYGON ((-47.11019 -14.67150, -47.11607 -14.6..."
5562,5222302,GO,Vila Propício,"POLYGON ((-48.75124 -14.90461, -48.75196 -14.9..."


In [3]:
ubs = pd.read_sql("SELECT cnes, UF, name, ST_AsText(geometry) AS geometry FROM ubs;", con = connection)

ubs['geometry'] = ubs['geometry'].map(geometrify)
ubs = gpd.GeoDataFrame(ubs)
# Quantidade de Unidades Básicas de Saúde por Unidade Federativa
ubs_uf = ubs[['cnes', 'UF']].groupby('UF').agg('count').rename(columns = {'cnes': 'count'}).reset_index()
ubs.head()

,cnes,UF,name,geometry
0,108,PE,USF ALTO DOS INDIOS,POINT (-35.03210 -8.28389)
1,116,PE,USF CHARNECA II,POINT (-35.02819 -8.28353)
2,124,PE,USF SAO FRANCISCO I,POINT (-35.03500 -8.28700)
3,132,PE,USF ROSARIO,POINT (-35.03210 -8.28389)
4,140,PE,USF JUSSARAL,POINT (-35.03500 -8.28700)
...,...,...,...,...
39645,9999078,MG,PRBE PRESIDIO DE BOA ESPERANCA,POINT (-45.55373 -21.09526)
39646,9999280,PB,EMAD II SAO JOAO DO RIO DO PEIXE,POINT (-38.44900 -6.72900)
39647,9999396,BA,UNIDADE BASICA DE SAUDE IRIS LOPES,POINT (-39.59576 -16.31248)
39648,9999493,PR,UNIDADE DE SAUDE SANTAREM,POINT (-49.29147 -25.67153)


ben_bas: Benefício Básico - R$ 89,00 \
ben_var: Benefício Variável - R$ 41,00 \
ben_bvj: Benefício Variável Jovem - R$ 48 \
ben_bvn: Benefício Variável Nutriz (criança de até 6 meses) \
ben_bvg: Benefício Variável à Gestante \
ben_bsp: Benefício para a Superação da Extrema Pobreza

In [83]:
cursor.execute('use geo_analysis;')
bolsa = pd.read_sql("SELECT * FROM bolsa_familia;", con = connection)

bolsa_ano = bolsa.drop(columns = ['id', 'y_m'])
functions = {x: 'mean' for x in bolsa_ano}
functions.update({'UF': 'first'})
functions.pop('ibge')

bolsa_ano = bolsa_ano.groupby('ibge').agg(functions)
bolsa_estado = bolsa_ano.groupby('UF').agg('mean')
bolsa_estado.head()

,qtd_ben_bas,qtd_ben_var,qtd_ben_bvj,qtd_ben_bvn,qtd_ben_bvg,qtd_ben_bsp
UF,,,,,,
AC,3877.943182,7202.352273,1015.818182,74.750000,174.367424,2749.840909
AL,3805.836601,4870.082516,696.629902,53.713235,136.162582,1872.531863
AM,6079.514785,11245.592742,1555.709677,91.701613,271.803763,3217.876344
AP,4458.437500,8034.916667,1123.625000,69.623958,171.572917,2593.250000
BA,4208.326139,4724.484213,722.635092,50.896483,116.331934,2146.903677


In [77]:
cursor.close()
connection.close()

In [108]:
#brazil.merge(bolsa_ano, on = '')
bolsa_ano

,UF,qtd_ben_bas,qtd_ben_var,qtd_ben_bvj,qtd_ben_bvn,qtd_ben_bvg,qtd_ben_bsp
ibge,,,,,,,
110001,RO,1243.250000,2370.333333,269.583333,25.916667,41.916667,237.416667
110002,RO,2739.750000,6213.666667,669.666667,68.333333,117.333333,449.250000
110003,RO,131.083333,301.083333,29.083333,1.909091,4.166667,42.916667
110004,RO,2263.916667,4799.416667,566.750000,58.416667,111.583333,877.500000
110005,RO,530.166667,1146.416667,132.916667,15.000000,23.916667,39.166667
...,...,...,...,...,...,...,...
522200,GO,257.916667,691.250000,74.500000,7.666667,12.333333,82.833333
522205,GO,758.833333,1078.666667,129.500000,11.000000,29.583333,107.583333
522220,GO,513.833333,791.583333,96.000000,9.333333,16.916667,48.166667


In [ ]:
import folium
m = folium.Map(location=[-15, -53], zoom_start=4, tiles='CartoDB dark_matter')

# Requer geopandas >= 1.10
brazil.explore(
    m = m,
    column = 'name',
    categorical = True,
    cmap = 'magma', 
    legend = False,
    tooltip = 'name',
    #scheme = 'BoxPlot',
    tooltip_kwds = dict(labels=False),
    k = 30, # Number of classes (5)
    name = 'Municípios'
)

# ubs.explore(
#     m = m,
#     color = 'green',
#     marker_kwds = dict(radius = 2, fill = True), # 2px
#     tooltip = 'name',
#     tooltip_kwds = dict(labels = False),
#     name = 'Unidades Básicas de Saúde'
# )

"""
marker_type: folium.Circle, folium.CircleMarker, folium.Marker
marker_kwds: {radius = 2, fill = True, icon = folium.map.Icon, draggable = False}
style_kwds: {stroke, color, weight, fill, fillColor, fillOpacity}


Schemes: 'BoxPlot', 'EqualInterval', 'FisherJenks', 'FisherJenksSampled', 'HeadTailBreaks'
         'JenksCaspall', 'JenksCaspallForced', 'JenksCaspallSampled', 'MaxP', 'MaximumBreaks'
         'NaturalBreaks', 'Quantiles', 'Percentiles', 'StdMean', 'UserDefined'

Layers : "OpenStreetMap", "Stamen Terrain", “Stamen Toner", “Stamen Watercolor" "CartoDB positron", “CartoDB dark_matter"
"""
folium.TileLayer('OpenStreetMap', control = True).add_to(m)
folium.TileLayer('Stamen Toner', control = True).add_to(m)
folium.LayerControl().add_to(m)
m

In [98]:
states = brazil.copy()
states['geometry'] = states.buffer(0)
states = states[['UF', 'geometry']].dissolve(by = 'UF', as_index = False)
states = states.merge(ubs_uf, on = 'UF')
states.head()

,UF,geometry,count
0,AC,"POLYGON ((-69.42758 -10.93700, -69.44967 -10.9...",241
1,AL,"POLYGON ((-37.19495 -9.89763, -37.22625 -9.899...",867
2,AM,"POLYGON ((-68.21979 -9.24215, -68.63703 -9.053...",579
3,AP,"POLYGON ((-51.26526 -0.19511, -51.28914 -0.227...",122
4,BA,"MULTIPOLYGON (((-42.33081 -15.07951, -42.33685...",3797


In [ ]:
# Infelizmente a legenda fica menos visível no tema escuro
m = folium.Map(location=[-15, -53], zoom_start=4, tiles='OpenStreetMap')

states.explore(
    m = m,
    cmap = 'summer_r', 
    legend = True,
    column = 'count',
    categorical = False,
    tooltip = 'UF',
    tooltip_kwds = dict(labels = False),
    k = 20, # Number of classes (5)
    name = 'Estados'
)
folium.TileLayer('CartoDB dark_matter', control = True).add_to(m)
folium.TileLayer('Stamen Toner', control = True).add_to(m)
folium.LayerControl().add_to(m)
m

In [ ]:
# HeatMap bem inútil, dê zoom no Norte

from folium import plugins

m = folium.Map(location = [-15, -53], tiles='Cartodb dark_matter', zoom_start = 4)
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in ubs.geometry]

plugins.HeatMap(heat_data).add_to(map)
m

In [ ]:
# Abandonado por enquanto

base = brazil.plot(figsize = (10,10), color = 'none', edgecolor = 'gainsboro', zorder = 1)
base.set_axis_off()
ubs.plot(ax = base, legend = False, zorder = 2, markersize = 2)

In [ ]:
# Guardado para emergências

m = folium.Map(location=[-15, -53], zoom_start=4, tiles='CartoDB dark_matter')

for _, r in brazil.iterrows():
    geo_j = gpd.GeoSeries(r['geometry']).to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(r['name']).add_to(geo_j)
    geo_j.add_to(m)
m